In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# To load the environment variable defined in the .env file
from dotenv import load_dotenv
load_dotenv();

In [3]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.calibration import CalibrationDisplay
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc

import src.data.NHLDataManager as DataManager
import src.features.build_features as FeaturesManager

In [7]:
seasons_year = [2015, 2016, 2017, 2018]
features_data_df = FeaturesManager.build_features(seasons_year)
features_data_df.drop(columns=['Team', 'Shooter', 'Goalie', 'Strength'], inplace=True)

features_data_df.sample(10)

,Game ID,Event Index,Period,Type,Shot Type,X,Y,Last event type,Last event X,Last event Y,...,Last event st_Y,Shot distance,Shot angle,Is Goal,Is Empty,Game seconds,Last event angle,Rebound,Change in Shot Angle,Speed From Previous Event
275380,2019020712,129,2,SHOT,Wrist Shot,-36.0,-1.0,Faceoff,-69.0,-22.0,...,22.0,53.009433,1.080924,0,0,1559,47.726311,0,0.000000,1.504431
88320,2017020213,222,3,SHOT,Wrist Shot,-52.0,-31.0,Missed Shot,35.0,-14.0,...,14.0,48.270074,39.957549,0,0,2457,6.441600,0,0.000000,9.849485
87972,2017020208,144,2,SHOT,Wrist Shot,71.0,32.0,Shot,68.0,-15.0,...,-15.0,36.715120,60.642246,0,0,1724,-35.537678,1,96.179924,11.773912
159833,2018020090,320,3,SHOT,Wrist Shot,51.0,-1.0,Shot,-69.0,22.0,...,22.0,38.013156,-1.507436,0,0,3133,7.926927,1,9.434362,8.145619
211511,2018020936,246,3,SHOT,Slap Shot,58.0,27.0,Shot,45.0,-14.0,...,-14.0,41.109610,41.054814,0,0,2646,-17.650124,1,58.704938,5.376453
45877,2016020756,188,2,SHOT,Wrist Shot,-59.0,-34.0,Faceoff,69.0,-22.0,...,22.0,45.343136,48.576334,0,0,2074,7.926927,0,0.000000,10.713439
23920,2016020393,241,3,SHOT,Slap Shot,-57.0,-10.0,Hit,-95.0,-31.0,...,31.0,33.526109,17.354025,0,0,2821,100.954063,0,0.000000,43.416587
42522,2016020702,63,1,SHOT,Tip-In,59.0,31.0,Hit,-8.0,-40.0,...,-40.0,43.139309,45.939191,0,0,665,-22.409837,0,0.000000,4.881086
2774,2016020047,119,2,GOAL,Wrist Shot,-63.0,-9.0,Faceoff,-69.0,-22.0,...,22.0,27.513633,19.093492,1,0,1355,47.726311,0,0.000000,4.772607
120266,2017020727,273,3,SHOT,Wrist Shot,71.0,-24.0,Hit,49.0,-37.0,...,-37.0,30.000000,-53.130102,0,0,3195,-42.768825,0,0.000000,1.344940
